#### Importing libraries

In [11]:
import requests
import pandas as pd
import numpy as np
import pickle
kmeans = pickle.load(open('kmeans_8_cluster.p', 'rb'))

#### getting my playlists

In [2]:
top_100_songs = pd.read_csv('top_100_songs.csv')
playlist_8_cluster = pd.read_csv('playlist_8_cluster.csv')

#### connect to spotify

In [3]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
secrets_file = open("../secrets.txt","r")

In [5]:
string = secrets_file.read()

In [6]:
string.split('\n')

['clientid:e9c8174323214a1d84247cc1e2018d9d',
 'clientsecret:11813fb826984d6ab23b42023d4a5b28']

In [7]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

### create a sample recommender

In [9]:
track = input("Enter a song that you love: ")

Enter a song that you love: simarik


In [47]:
if top_100_songs['title'].str.lower().str.contains(str(track).lower()).any() == True:
        print('here what I found for you! ')
        print(top_100_songs['title'].sample())
else:
        results = sp.search(q="track:" + track, type="track")
        song_uri = results['tracks']['items'][0]['uri'].split(':')[2]
        song_features = sp.audio_features(song_uri)
        new = pd.DataFrame.from_dict(song_features)
        y_pred = new.drop(columns=['type', 'id' , 'uri', 'track_href', 'analysis_url'])
        look_for = kmeans.predict(y_pred)
        print('Here what I found for you !!!')
        print(playlist_8_cluster.loc[playlist_8_cluster.clusters == look_for[0],['title','artist','url']].sample(1))
            

Here what I found for you !!!
                              title        artist  \
1549  LEGO Ninjago The WEEKEND WHIP  The Spinners   

                                                        url  
1549  https://open.spotify.com/track/1Q6h73UKeNO0AtA1TRS3tL  


In [16]:
# from sklearn.preprocessing import StandardScaler
# target = StandardScaler().fit_transform(y_pred)

# # Now, all features will have the same weight.
# pd.DataFrame(target,columns=y_pred.columns).head()